In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import random
import os
import PYCLASS2_air_Copy1
from torch.utils.data import Dataset, DataLoader
#from torchvision import models


# PYCLASS IMPORT

In [2]:
sound_train = PYCLASS2_air_Copy1.my_datset2(train = True)
                         

sound_test = PYCLASS2_air_Copy1.my_datset2(train = False)

In [3]:
batch_size = 25

train_loader= DataLoader(dataset=sound_train, batch_size=batch_size, shuffle=False ,num_workers=1)

test_loader = DataLoader(dataset=sound_test,  batch_size=batch_size, shuffle=False, num_workers=1)

In [4]:
print(sound_train.train_data.shape)
print(sound_train.train_label.shape)
print(sound_train.train_data.dtype)
print(sound_test.test_data.shape)
print(sound_test.test_label.shape)

torch.Size([898, 1, 10, 20])
torch.Size([898])
torch.float32
torch.Size([102, 1, 10, 20])
torch.Size([102])


In [5]:
sound_test.test_label

tensor([1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1,
        2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2,
        1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2,
        1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1,
        1, 2, 2, 1, 1, 2])

In [6]:
sound_train.train_data[0][0][0]

tensor([-142.1522,  170.4252,   -7.8784,   -2.5154,  -31.1027,   15.8497,
          -5.4729,    5.4637,  -11.9064,  -14.5147,   -9.9733,    0.4618,
         -14.3489,   -4.4176,   -8.8174,   10.0748,   -6.8559,  -19.3608,
         -10.5782,    4.4927])

# DEFINE MODEL

In [7]:
class CNN2(nn.Module):
    def __init__(self):
        
        super(CNN2, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 1,out_channels=16,kernel_size = 3),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size = 3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout2d(0.5),
            nn.Conv2d(in_channels = 32,out_channels = 64,kernel_size = 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            #nn.MaxPool2d(2,2)
            
        )
        
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*6*1,1000),
            nn.ReLU(),
            nn.Linear(1000,500),
            nn.ReLU(),
            nn.Linear(500,300),
            nn.ReLU(),
            nn.Linear(300,100),
            nn.ReLU(),
            nn.Linear(100,10),
            nn.ReLU(),
            nn.Linear(10,3),
            nn.Softmax()
            #nn.Softmax(dim=1)
        )       
        
    def forward(self,x):
        print(x.data.shape)
        out = self.layer(x)
        out = out.view(out.shape[0],-1)
        out = self.fc_layer(out)

        return out

In [8]:
model2 = CNN2().cuda()

In [9]:
loss = nn.CrossEntropyLoss()

optimizer = optim.Adam(model2.parameters())

# TRAIN MODEL

In [12]:
num_epochs = 500

In [13]:
save_path = '/home/libedev/mute/mute-hero/download/dataset/model2/'
model_path = save_path + 'model2.pkl'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [14]:
if os.path.isfile(model_path):
    model2.load_state_dict(torch.load(model_path))
    print("Model Loaded!")

else:
    
    for epoch in range(num_epochs):

        total_batch = len(sound_train) // batch_size

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            X = batch_images.cuda()
            Y = batch_labels.cuda()

            pre = model2(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 10 == 0:
                print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                     %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

    if not os.path.isfile(model_path):
        print("Model Saved!")
        torch.save(model2.state_dict(), model_path)

torch.Size([25, 1, 10, 20])


/home/libedev/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [1/500], lter [10/35], Loss: 0.7570
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [1/500], lter [20/35], Loss: 0.9286
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [1/500], lter [30/35], Loss: 0.7756
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.

torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [8/500], lter [30/35], Loss: 0.6714
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([23, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [9/500], lter [10/35], Loss: 0.7512
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 1

torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [16/500], lter [10/35], Loss: 0.5516
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [16/500], lter [20/35], Loss: 0.5916
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [16/500], lter [30/35], Loss: 0.6715
torch.Size([25, 1, 10, 20])
tor

...

torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [467/500], lter [10/35], Loss: 0.5545
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [467/500], lter [20/35], Loss: 0.5914
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [467/500], lter [30/35], Loss: 0.5609
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])


torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [474/500], lter [30/35], Loss: 0.5914
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([23, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [475/500], lter [10/35], Loss: 0.5914
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 

torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([23, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [482/500], lter [10/35], Loss: 0.5515
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [482/500], lter [20/35], Loss: 0.6027
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 

torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [489/500], lter [10/35], Loss: 0.6315
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [489/500], lter [20/35], Loss: 0.5789
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [489/500], lter [30/35], Loss: 0.5914
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])


torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [496/500], lter [30/35], Loss: 0.7514
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([23, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
Epoch [497/500], lter [10/35], Loss: 0.6314
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 

# inception_transfer_learning

In [16]:
model = models.inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /home/libedev/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth
22.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

58.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

93.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable

In [17]:
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [18]:
model.aux_logits = False

fine_tune = False

if not fine_tune:
    for parameter in model.parameters():
        parameter.requires_grad = False

In [19]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 100),
    nn.Linear(100,11)
)

In [20]:
model = model.cuda()

In [21]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [10]:
num_epochs = 500

In [11]:
save_path = '/home/libedev/mute/mute-hero/download/dataset/model2/'
model_path = save_path + 'model.pkl'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [15]:
if os.path.isfile(model_path):
    model2.load_state_dict(torch.load(model_path))
    print("Model Loaded!")

else:
    
    for epoch in range(num_epochs):

        total_batch = len(sound_train) // batch_size

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            X = batch_images.cuda()
            Y = batch_labels.cuda()

            pre = model2(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 10 == 0:
                print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                     %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

    if not os.path.isfile(model_path):
        print("Model Saved!")
        torch.save(model2.state_dict(), model_path)

Model Loaded!


# TEST MODEL

In [16]:
model2.eval()

correct = 0
total = 0

for sound, labels in test_loader:
    
    sound = sound.cuda()
    outputs = model2(sound)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of test sound: %f %%' % (100 * float(correct) / total))

torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([2, 1, 10, 20])
Accuracy of test sound: 100.000000 %


In [17]:
for sound, labels in test_loader:
    
    sound = sound.cuda()
    outputs = model2(sound)
    
    _, predicted = torch.max(outputs.data, 1)
    
    print(predicted)

torch.Size([25, 1, 10, 20])
tensor([1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1,
        2], device='cuda:0')
torch.Size([25, 1, 10, 20])
tensor([2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1,
        2], device='cuda:0')
torch.Size([25, 1, 10, 20])
tensor([1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1,
        2], device='cuda:0')
torch.Size([25, 1, 10, 20])
tensor([2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2,
        1], device='cuda:0')
torch.Size([2, 1, 10, 20])
tensor([1, 2], device='cuda:0')


In [18]:
model2.eval()

correct = 0
total = 0

for sound, labels in test_loader:
    
    sound = sound.cuda()
    outputs = model2(sound)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
        
    tp = (correct * predicted).sum().to(torch.float32)
    tn = ((1 - correct) * (1 - predicted)).sum().to(torch.float32)
    fp = ((1 - correct) * predicted).sum().to(torch.float32)
    fn = (correct * (1 - predicted)).sum().to(torch.float32)
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    
print('Accuracy of test sound: %f %%' % (100 * float(correct) / total))
print('precision :',precision)
print('recall :',recall)
print('F1_SCORE :',f1)

torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([25, 1, 10, 20])
torch.Size([2, 1, 10, 20])
Accuracy of test sound: 100.000000 %
precision : tensor(102., device='cuda:0')
recall : tensor(1.5000, device='cuda:0')
F1_SCORE : tensor(2.9565, device='cuda:0')
